In [1]:
#import useful libriaries.
import random
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import utils
from tensorflow.python.ops.math_ops import reduce_prod
from sklearn.model_selection import train_test_split
from LoadFaceData import load_dataset, resize_image, IMAGE_SIZE

In [2]:
class Dataset:
    def __init__(self, path_name):
        # Test dataset
        self.train_images = None
        self.train_labels = None

        # Validation dataset
        self.valid_images = None
        self.valid_labels = None

        # Test dataset
        self.test_images = None
        self.test_labels = None

        # The path of dataset
        self.path_name = path_name

        # The shape of input data of CNN or LSTM
        self.input_shape = None
        
        #The number of dataset label.
        self.nb_classes = None

    # Load the dataset and preprocess pictures into suitable dim for CNN or LSTM
    def load(self, img_rows=IMAGE_SIZE, img_cols=IMAGE_SIZE,
             img_channels=3):
        
        # Define some useful variables
        image = []
        labels = []
        face_num = []
        
        #Load basical dataset and return it into variables 
        images, labels, face_num = load_dataset(self.path_name)
        self.nb_classes = face_num
        
        # Use the split function to separate the basical picture into different array for furture test and validation
        train_images, valid_images, train_labels, valid_labels = train_test_split(images, labels, test_size=0.3,
                                                                                  random_state=random.randint(0, 100))
        _, test_images, _, test_labels = train_test_split(images, labels, test_size=0.5,
                                                          random_state=random.randint(0, 100))

        # if the data dim is 'th'，the data qualitu is ：channels,rows,cols，if not:rows,cols,channels
        # Reshape all dataset including training, validation, test images into a special shape according the keras specification.
        if K.image_data_format() == 'channels_first':
            train_images = train_images.reshape(train_images.shape[0], img_channels, img_rows, img_cols)
            valid_images = valid_images.reshape(valid_images.shape[0], img_channels, img_rows, img_cols)
            test_images = test_images.reshape(test_images.shape[0], img_channels, img_rows, img_cols)
            self.input_shape = (img_channels, img_rows, img_cols)
        else:
            train_images = train_images.reshape(train_images.shape[0], img_rows, img_cols, img_channels)
            valid_images = valid_images.reshape(valid_images.shape[0], img_rows, img_cols, img_channels)
            test_images = test_images.reshape(test_images.shape[0], img_rows, img_cols, img_channels)
            self.input_shape = (img_rows, img_cols, img_channels)

            # Show the number of training, valid, test image.
            print(train_images.shape[0], 'train samples')
            print(valid_images.shape[0], 'valid samples')
            print(test_images.shape[0], 'test samples')

            # Convert the labels into one-hot code.
            train_labels = utils.to_categorical(train_labels, self.nb_classes)
            valid_labels = utils.to_categorical(valid_labels, self.nb_classes)
            test_labels = utils.to_categorical(test_labels, self.nb_classes)
            
            # Change the data into float type.
            train_images = train_images.astype('float32')
            valid_images = valid_images.astype('float32')
            test_images = test_images.astype('float32')

            # Nomalization all data in to 0 - 1.
            train_images /= 255
            valid_images /= 255
            test_images /= 255
            
            # Assign all image and label data to class variables
            self.train_images = train_images
            self.valid_images = valid_images
            self.test_images = test_images
            self.train_labels = train_labels
            self.valid_labels = valid_labels
            self.test_labels = test_labels


# LSTM model
class Model:
    def __init__(self):
        self.model = None
 
    # Built a model
    def build_model(self, dataset, nb_classes):
       
        # Use a list to discribe a LSTM layers.
        # Here contains 3 layers, first layter contain 256 cells, second layer contain 128 cells, third layer 64 cells.
        # The timestep is 3(the BGR channel number), batch-size is size of grey picture(64*64).
        # Return a sequences tensor.
        # Each LSTM layer followed by a dropout layer, which used to decrease over-fit.Here it give up 20% cells.
        # Use a Flatten layer to conbine all layers output.with a 256 neurons dense layer to train the net.
        # Use a dense layer to output a face number classes.
        LSTM_layers = [
            
            LSTM(256, input_shape = (3, IMAGE_SIZE*IMAGE_SIZE), return_sequences = True),
            Dropout(0.2),
            
            LSTM(128, input_shape = (3, IMAGE_SIZE*IMAGE_SIZE), return_sequences = True),
            Dropout(0.2),
            
            LSTM(64, input_shape = (3, IMAGE_SIZE*IMAGE_SIZE), return_sequences = True),
            Dropout(0.2),
            
            LSTM(32),
            Dropout(0.2),
            
            Flatten(),
            #Dense(256, activation = tf.nn.relu),
            #Dropout(0.25),
            Dense(128, activation = tf.nn.relu),
            Dropout(0.25),
            Dense(128, activation = tf.nn.relu),
            Dropout(0.25),
            
            Dense(nb_classes, activation = tf.nn.softmax)
              
        ]
        
        self.model = Sequential(LSTM_layers)
        
        self.model.summary()
           
        
    # Train the LSTM model
    def train(self, dataset, batch_size=20, nb_epoch=20, data_augmentation=True):
        
        # Define a SGD optimizer.
        sgd = SGD(learning_rate=0.0007, decay=1e-6,
                  momentum=0.9, nesterov=True)
        
        # Define a Adam optimazer
        adam = Adam(learning_rate = 0.0007, decay = 1e-6)
        
        # compile a model, loss funtion use the categorical_crossentropy.
        self.model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])  # 完成实际的模型配置工作

        # Reshape the dataset shape into the LSTM required.The shape here is [image numbers, channels = 3, imageshape = 64*64]
        dataset.train_images = dataset.train_images.reshape(dataset.train_images.shape[0], dataset.train_images.shape[3], 
                                                                dataset.train_images.shape[1]*dataset.train_images.shape[2]) 
        dataset.valid_images = dataset.valid_images.reshape(dataset.valid_images.shape[0], dataset.valid_images.shape[3], 
                                                                dataset.valid_images.shape[1]*dataset.valid_images.shape[2])
        dataset.test_images = dataset.test_images.reshape(dataset.test_images.shape[0], dataset.test_images.shape[3], 
                                                                dataset.test_images.shape[1]*dataset.test_images.shape[2])
        # Train and fit a model    
        self.model.fit(dataset.train_images, dataset.train_labels,
                                     validation_data=(dataset.valid_images, dataset.valid_labels),
                                     steps_per_epoch=len(dataset.train_images),
                                     epochs=nb_epoch)
    
    # Define a model parameter store path 
    MODEL_PATH = './model'
    
    # Save the model parameter in the defined path
    def save_model(self, file_path=MODEL_PATH):
        self.model.save(file_path)
    
    #Load a model from the defined path
    def load_model(self, file_path=MODEL_PATH):
        self.model = load_model(file_path)

    # Evaluate a model from the defined path using the dataset
    def evaluate(self, dataset):
        score = self.model.evaluate(dataset.test_images, dataset.test_labels, verbose=1)
        print("%s: %.2f%%" % (self.model.metrics_names[1], score[1] * 100))
    
    # Recognize the face
    def face_predict(self, image):
   
        # Accordinf the Dim to reshape the image shape.
        if K.image_data_format() == 'channels_first':
            image = resize_image(image) 
            image = image.reshape((1, 3, IMAGE_SIZE, IMAGE_SIZE))
        else:
            image = resize_image(image)
            image = image.reshape((1, IMAGE_SIZE, IMAGE_SIZE, 3))
         
        # Reshape the shape in the same shape as LSTM input.
        image = image.reshape(image.shape[0], image.shape[3], image.shape[1]*image.shape[2])    
            
        # Float and normalize the input.
        image = image.astype('float32')
        image /= 255
        #print(image.shape)
        
        # Print the probability of name response the present face.
        result_probability = self.model.predict(image)
        print('result:', result_probability, max(result_probability[0]))

        # Output the one-hot code and change it into the name od user.
        result = self.model.predict(image)
        result = np.argmax(result, axis=-1)

        return max(result_probability[0]),result[0]



if __name__ == '__main__':
    dataset = Dataset('./Facial data')
    dataset.load()
    model = Model()
    model.build_model(dataset, dataset.nb_classes)
    model.train(dataset)
    model.save_model(file_path='./model')
    model.evaluate(dataset)


(787, 64, 64, 3)
['Jiahao Wang', 'Xingda Zhou', 'Zhongen Qin', 'Yuxi Wu', 'Yuan Li', 'Jufeng Yang']
face_num: 6
contrast_table: {0: 'Jiahao Wang', 1: 'Xingda Zhou', 2: 'Zhongen Qin', 3: 'Yuxi Wu', 4: 'Yuan Li', 5: 'Jufeng Yang'}
550 train samples
237 valid samples
394 test samples
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 3, 256)            4457472   
_________________________________________________________________
dropout (Dropout)            (None, 3, 256)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 3, 128)            197120    
_________________________________________________________________
dropout_1 (Dropout)          (None, 3, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 3,

accuracy: 94.92%
